In [8]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pickle
import numpy as np

# Load model and encoders
with open('rf_model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('label_encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

# Features required by model
features = ['Visibility', 'Humidity', 'Wind_Speed', 'Temperature', 'Pressure']

# GUI setup
root = tk.Tk()
root.title("🌤 Weather Prediction System")
root.geometry("450x400")
root.resizable(False, False)

style = ttk.Style()
style.configure("TLabel", font=("Segoe UI", 10))
style.configure("TEntry", font=("Segoe UI", 10))
style.configure("TButton", font=("Segoe UI", 10, "bold"))

entries = {}

def predict_weather():
    try:
        input_data = {}

        for feature in features:
            val = entries[feature].get()
            if val == "":
                raise ValueError(f"{feature} cannot be empty.")
            
            if feature in encoders:
                le = encoders[feature]
                if val not in le.classes_:
                    le.classes_ = np.append(le.classes_, val)
                input_data[feature] = le.transform([val])[0]
            else:
                input_data[feature] = float(val)

        # Create DataFrame and ensure correct column order
        user_df = pd.DataFrame([input_data])
        user_df = user_df[model.feature_names_in_]

        # Predict
        prediction = model.predict(user_df)[0]
        messagebox.showinfo("✅ Prediction Result", f"Predicted Weather Condition: {prediction}")
    
    except ValueError as ve:
        messagebox.showwarning("⚠ Input Error", str(ve))
    except Exception as e:
        messagebox.showerror("❌ Error", f"Something went wrong:\n{str(e)}")

# Input fields
for i, feature in enumerate(features):
    ttk.Label(root, text=feature + ":").grid(row=i, column=0, padx=10, pady=8, sticky='e')

    if feature in encoders:
        # Use dropdown for encoded categorical features
        combo = ttk.Combobox(root, values=list(encoders[feature].classes_), state="readonly", font=("Segoe UI", 10))
        combo.grid(row=i, column=1, padx=10, pady=8)
        entries[feature] = combo
    else:
        entry = ttk.Entry(root, width=30)
        entry.grid(row=i, column=1, padx=10, pady=8)
        entries[feature] = entry

# Predict button
predict_btn = ttk.Button(root, text="🔍 Predict Weather", command=predict_weather)
predict_btn.grid(row=len(features), column=0, columnspan=2, pady=30)

root.mainloop()
